# Week 4: Guardrails and Output Controls

**Scope:** Study, test, and analyze guardrails, safety, and output controls in a RAG system.

**Learning Objectives:**
1. Understand common risks in LLM outputs (hallucinations, ungrounded answers, restricted topics)
2. Implement simple rule-based guardrail system (no ML, no external APIs)
3. Add validation steps to reduce hallucinations and enforce grounding
4. Document security considerations at intern awareness level
5. **Demonstrate real LLM control**: Show that guardrails actually block or allow LLM calls

**Approach:** Research-oriented experiments similar to Week 1 — test different thresholds, record decisions, analyze trade-offs.

**Key Feature:** This notebook demonstrates guardrails controlling a REAL LLM (Ollama), showing when LLM calls are blocked vs allowed. Experiments A and B make this behavior observable and testable.

In [42]:
# Cell 1: IMPORTS ONLY

import numpy as np
import pandas as pd
from pathlib import Path
from typing import Dict, List, Tuple, Optional
from sentence_transformers import SentenceTransformer
import faiss
from langchain_ollama import OllamaLLM
from langchain_core.prompts import PromptTemplate

## Understanding Risks in RAG Systems

Before building guardrails, we need to understand what can go wrong:

### 1. Hallucinations
- **What it is:** LLM generates information that sounds plausible but isn't in the retrieved context
- **Why it happens:** LLMs are trained to be helpful and fluent, even without evidence
- **Risk:** Users trust incorrect information

### 2. Ungrounded Answers
- **What it is:** LLM answers when no relevant context was retrieved
- **Why it happens:** LLM doesn't know it lacks information
- **Risk:** Answers based on training data, not our knowledge base

### 3. Restricted Topics
- **What it is:** Questions about sensitive information (PII, out-of-domain)
- **Why it matters:** We should only answer questions within our domain
- **Risk:** Privacy violations, incorrect domain answers

### 4. Information Leakage
- **What it is:** Accidental exposure of sensitive data in responses
- **Why it matters:** Even if retrieved, some information shouldn't be shared
- **Risk:** Privacy and security violations

## Guardrail Design

We implement a **simple, rule-based system** that:

1. **Checks retrieval quality** before generation:
   - Top-1 similarity score must exceed a threshold
   - Gap between top-1 and top-2 must be large enough (reduces ambiguity)
   - Must have at least one retrieved result

2. **Checks query content** before retrieval:
   - Blocks questions about PII (names, emails, SSNs)
   - FIX #3: Out-of-domain queries are handled by retrieval quality gating only (Option A - simplest)
     - No keyword-based out-of-domain checks
     - Low similarity scores naturally catch out-of-domain queries

3. **Validates generated output** after generation:
   - Rejects empty answers
   - Flags uncertain language ("I think", "probably", "might")

**Why rule-based?**
- Explainable: we can see exactly why a decision was made
- Testable: we can test with different thresholds
- Reproducible: same inputs always give same decisions
- No external dependencies: works offline

In [43]:
# Cell 2: GUARDRAIL FUNCTIONS ONLY

# Refusal reason codes
REFUSAL_REASONS = {
    "NO_CONTEXT": "No relevant context retrieved (similarity too low)",
    "AMBIGUOUS_RETRIEVAL": "Retrieval results are ambiguous (top-1 and top-2 too close)",
    "EMPTY_RETRIEVAL": "No documents retrieved",
    "PII_DETECTED": "Query asks for personally identifiable information",
    "OUT_OF_DOMAIN": "Query is outside our knowledge domain",
    "EMPTY_ANSWER": "Generated answer is empty",
    "UNCERTAIN_LANGUAGE": "Generated answer contains uncertain language",
}


def check_pii(query: str) -> bool:
    """
    Simple PII detection: check for common PII patterns.
    
    This is a basic implementation for learning purposes.
    In production, you'd use more sophisticated methods.
    """
    pii_keywords = [
        "email", "phone", "ssn", "social security",
        "credit card", "passport", "driver's license",
        "address", "zip code", "date of birth",
    ]
    query_lower = query.lower()
    return any(keyword in query_lower for keyword in pii_keywords)


# FIX #1: Removed check_out_of_domain() function
# We rely on retrieval quality gating instead:
# - If query is out-of-domain, retrieval will have low similarity
# - Low similarity triggers "NO_CONTEXT" refusal (simpler, more reliable)
# This makes the guardrail system simpler and more explainable.


def check_retrieval_quality(
    scores: np.ndarray,
    similarity_threshold: float = 0.30,
    ambiguity_gap: float = 0.05
) -> Tuple[bool, Optional[str]]:
    """
    Check if retrieval results meet quality criteria.
    
    Args:
        scores: Array of similarity scores (sorted descending)
        similarity_threshold: Minimum top-1 score required
        ambiguity_gap: Minimum gap between top-1 and top-2 scores
    
    Returns:
        (allowed, reason): True if retrieval is good enough, False with reason if not
    """
    # Empty retrieval
    if len(scores) == 0:
        return False, REFUSAL_REASONS["EMPTY_RETRIEVAL"]
    
    # Top-1 score too low
    top1_score = scores[0]
    if top1_score < similarity_threshold:
        return False, REFUSAL_REASONS["NO_CONTEXT"]
    
    # Ambiguous retrieval (top-1 and top-2 too close)
    if len(scores) >= 2:
        top2_score = scores[1]
        gap = top1_score - top2_score
        if gap < ambiguity_gap:
            return False, REFUSAL_REASONS["AMBIGUOUS_RETRIEVAL"]
    
    return True, None


def check_generated_answer(answer: str) -> Tuple[bool, Optional[str]]:
    """
    Check if generated answer passes post-generation validation.
    
    FIX #6: Do not apply uncertain-language checks to INSUFFICIENT_CONTEXT.
    Only validate uncertain language when outcome would be "answer".
    
    Args:
        answer: Generated answer text
    
    Returns:
        (allowed, reason): True if answer is acceptable, False with reason if not
    """
    # Empty answer
    if not answer or len(answer.strip()) == 0:
        return False, REFUSAL_REASONS["EMPTY_ANSWER"]
    
    # FIX #6: INSUFFICIENT_CONTEXT is handled separately in generate_with_guardrails()
    # Do not apply uncertain-language checks here for INSUFFICIENT_CONTEXT
    if answer and "insufficient_context" in answer.lower():
        return True, None  # Valid outcome, handled separately
    
    # Uncertain language check (only for normal answers, not INSUFFICIENT_CONTEXT)
    uncertain_phrases = [
        "i think", "i believe", "probably", "might",
        "possibly", "perhaps", "maybe", "could be",
        "i'm not sure", "i don't know", "uncertain",
    ]
    answer_lower = answer.lower()
    for phrase in uncertain_phrases:
        if phrase in answer_lower:
            return False, REFUSAL_REASONS["UNCERTAIN_LANGUAGE"]
    
    return True, None


# FIX #1: LEGACY FUNCTION - DO NOT USE
# This function is kept for reference only. All experiments must use generate_with_guardrails() instead.
def apply_guardrails(
    query: str,
    retrieval_scores: np.ndarray,
    generated_answer: str,
    domain_keywords: List[str],
    similarity_threshold: float = 0.30,
    ambiguity_gap: float = 0.05,
) -> Dict:
    """
    LEGACY: This function is deprecated. Use generate_with_guardrails() instead.
    
    This function is kept for reference only and should not be used in experiments.
    generate_with_guardrails() is the single source of truth for guardrail decisions.
    """
    raise DeprecationWarning("apply_guardrails() is deprecated. Use generate_with_guardrails() instead.")


# LLM Integration for Real Generation
def create_grounded_prompt() -> PromptTemplate:
    """
    Create a strict grounding prompt that forces LLM to use ONLY retrieved context.
    
    This prompt explicitly tells the LLM:
    - Use ONLY the provided context
    - If answer is not in context, say "INSUFFICIENT_CONTEXT"
    - No guessing, no external knowledge
    """
    template = """You are a helpful assistant that answers questions using ONLY the provided context.

Context:
{context}

Question: {query}

Instructions:
1. Answer the question using ONLY information from the context above.
2. If the answer is not in the context, respond with exactly: "INSUFFICIENT_CONTEXT"
3. Do not use any external knowledge or make guesses.
4. If you are uncertain, respond with "INSUFFICIENT_CONTEXT"

Answer:"""
    
    return PromptTemplate(template=template, input_variables=["context", "query"])


# FIX #4: Separate mock vs real LLM paths explicitly
def mock_llm(prompt: str) -> str:
    """
    Mock LLM for baseline testing (no real LLM call).
    
    FIX #4: This provides explicit mock behavior separate from real LLM path.
    
    Args:
        prompt: Formatted prompt string (not used, but kept for interface consistency)
    
    Returns:
        Mock answer string
    """
    # Simple mock: return INSUFFICIENT_CONTEXT for most cases
    # This demonstrates guardrail behavior without requiring LLM
    return "INSUFFICIENT_CONTEXT"


def create_llm_fn(llm: Optional[OllamaLLM]) -> callable:
    """
    FIX #4: Create LLM callable function for explicit separation.
    
    Returns either a real LLM callable or mock_llm.
    
    Args:
        llm: OllamaLLM instance or None
    
    Returns:
        Callable that takes (prompt: str) -> str
    """
    if llm is None:
        return mock_llm
    else:
        # Return a lambda that calls the real LLM
        return lambda prompt: llm.invoke(prompt)


def generate_with_guardrails(
    query: str,
    retrieved_chunks: List[str],
    retrieval_scores: np.ndarray,
    llm_fn,  # FIX #4: Accept callable (call_llm or mock_llm) instead of llm object
    prompt_template: PromptTemplate,
    similarity_threshold: float = 0.30,
    ambiguity_gap: float = 0.05,
) -> Dict:
    """
    FIX #1: Single source of truth for guardrail decisions.
    
    This is the ONLY function that makes guardrail decisions.
    All experiments must call this function.
    
    FIX #4: Accepts llm_fn callable (either call_llm or mock_llm) for explicit separation.
    
    FIX #2: Returns explicit outcome field: "answer" | "insufficient_context" | "refusal"
    
    Args:
        query: User query string
        retrieved_chunks: List of retrieved document chunks
        retrieval_scores: Array of similarity scores (sorted descending)
        llm_fn: Callable that takes (prompt: str) -> str (either call_llm or mock_llm)
        prompt_template: PromptTemplate for formatting
        similarity_threshold: Minimum top-1 score required
        ambiguity_gap: Minimum gap between top-1 and top-2 scores
    
    Returns:
        Dictionary with:
        - outcome: str ("answer" | "insufficient_context" | "refusal")
        - allowed: bool (False for refusal/insufficient_context, True for answer)
        - stage: str ("pre" | "post" | "final")
        - reason: str | None (refusal reason if outcome="refusal")
        - answer: str | None (generated answer or INSUFFICIENT_CONTEXT)
        - llm_called: bool (whether LLM was actually invoked)
    """
    result = {
        "outcome": "refusal",  # FIX #2: Explicit outcome field
        "allowed": False,
        "stage": "pre",
        "reason": None,
        "answer": None,
        "llm_called": False,
    }
    
    # PRE-CHECK 1: PII Detection
    if check_pii(query):
        result["reason"] = REFUSAL_REASONS["PII_DETECTED"]
        result["stage"] = "pre"
        result["outcome"] = "refusal"
        return result
    
    # FIX #3: Removed out-of-domain keyword check (Option A - simplest)
    # Out-of-domain queries naturally have low similarity and are caught by retrieval quality gating
    
    # PRE-CHECK 2: Retrieval Quality
    retrieval_ok, retrieval_reason = check_retrieval_quality(
        retrieval_scores, similarity_threshold, ambiguity_gap
    )
    if not retrieval_ok:
        result["reason"] = retrieval_reason
        result["stage"] = "pre"
        result["outcome"] = "refusal"
        return result
    
    # All PRE-checks passed - we can call the LLM
    try:
        # FIX #4: Use llm_fn callable (explicit separation of mock vs real)
        result["llm_called"] = (llm_fn != mock_llm)  # True if real LLM, False if mock
        context = "\n\n".join(retrieved_chunks)
        prompt = prompt_template.format(context=context, query=query)
        generated_answer = llm_fn(prompt)  # Call the function (either real or mock)
        result["answer"] = generated_answer
    except Exception as e:
        # If LLM call fails, treat as refusal
        result["llm_called"] = True  # We attempted to call it
        result["answer"] = ""
        result["reason"] = f"LLM_UNAVAILABLE: {str(e)[:100]}"
        result["stage"] = "post"
        result["outcome"] = "refusal"
        result["allowed"] = False
        return result
    
    # FIX #2: Check for INSUFFICIENT_CONTEXT as first-class outcome
    if result["answer"] and "insufficient_context" in result["answer"].lower():
        result["outcome"] = "insufficient_context"
        result["allowed"] = False  # FIX #2: INSUFFICIENT_CONTEXT is not "allowed" but is expected
        result["stage"] = "final"
        result["reason"] = "INSUFFICIENT_CONTEXT"
        return result
    
    # POST-CHECK: Validate Generated Answer (only for normal answers, not INSUFFICIENT_CONTEXT)
    answer_ok, answer_reason = check_generated_answer(result["answer"])
    if not answer_ok:
        result["reason"] = answer_reason
        result["stage"] = "post"
        result["outcome"] = "refusal"
        result["allowed"] = False
        return result
    
    # All checks passed - normal answer
    result["outcome"] = "answer"
    result["allowed"] = True
    result["stage"] = "final"
    result["reason"] = None
    return result

---
## FIX #3: Out-of-Domain Handling (Option A - Simplest)

**Approach:** We removed keyword-based out-of-domain checks entirely.

**Rationale:**
- Out-of-domain queries naturally have low similarity scores with our knowledge base
- Retrieval quality gating (low similarity threshold) automatically catches them
- Simpler, more reliable, and explainable than keyword heuristics

**How it works:**
- Empty retrieval OR top1 < min_score OR gap < min_gap → outcome="refusal"
- No separate "out-of-domain" check needed

---
## FIX #5: LLM Reproducibility

**Approach:** We lock one explicit Ollama model for reproducibility.

**Model:** `OLLAMA_MODEL = "llama3.2:1b"`

**Why:**
- No dynamic model selection (no loops)
- Same model = same results across runs
- Clear error message if model unavailable

**Note:** Change `OLLAMA_MODEL` constant if you need a different model.

In [50]:
# Load embedding model and create a simple test index
model = SentenceTransformer("all-MiniLM-L6-v2")

# Simple test documents (RAG domain)
test_docs = [
    "Retrieval-Augmented Generation (RAG) combines retrieval with language models to improve accuracy.",
    "Vector databases store embeddings for fast similarity search.",
    "Chunking strategies affect retrieval quality in RAG systems.",
    "Embeddings convert text into dense vector representations.",
    "FAISS is a library for efficient similarity search.",
]

# Build index
embeddings = model.encode(test_docs, normalize_embeddings=True)
embeddings = np.array(embeddings, dtype="float32")

index = faiss.IndexFlatIP(embeddings.shape[1])
index.add(embeddings)

print(f"Index built with {index.ntotal} documents")
print(f"Embedding dimension: {embeddings.shape[1]}")

# FIX #5: Use ONE explicit Ollama model for reproducibility
# We lock one model for reproducibility (no dynamic model selection)
OLLAMA_MODEL = "llama3.2:1b"  # Explicit model choice - change if needed

llm = None
prompt_template = create_grounded_prompt()

try:
    llm = OllamaLLM(model=OLLAMA_MODEL)
    print(f"\n✅ LLM configured: Using model '{OLLAMA_MODEL}'")
    print("   (Model choice is explicit for reproducibility)")
except Exception as e:
    llm = None
    print(f"\n⚠️  LLM not available: Model '{OLLAMA_MODEL}' not found")
    print("   To use real LLM:")
    print(f"   1. Install Ollama: https://ollama.ai")
    print(f"   2. Pull the model: ollama pull {OLLAMA_MODEL}")
    print("\n   Notebook will use mock_llm() for testing - guardrails still function correctly!")

# FIX #4: Create LLM callable for explicit separation
llm_fn = create_llm_fn(llm)

Loading weights: 100%|██████████| 103/103 [00:00<00:00, 2175.38it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Index built with 5 documents
Embedding dimension: 384

✅ LLM configured: Using model 'llama3.2:1b'
   (Model choice is explicit for reproducibility)


---
## Experiment 2: Define Test Queries

We need diverse test queries to evaluate guardrails:
- **Good queries:** Should pass (in-domain, good retrieval)
- **Low similarity:** Should be refused (no relevant context)
- **Ambiguous retrieval:** Should be refused (top-1 and top-2 too close)
- **Out-of-domain:** Should be refused (cooking, weather, etc.)
- **PII queries:** Should be refused (privacy concerns)

In [51]:
# Test queries with expected outcomes
test_queries = [
    # Good queries (should pass)
    ("What is RAG?", "GOOD", "In-domain, should retrieve well"),
    ("How do vector databases work?", "GOOD", "In-domain, should retrieve well"),
    
    # Low similarity (should be refused)
    ("What is the capital of France?", "LOW_SIMILARITY", "Out-of-domain, low similarity"),
    ("How do I cook pasta?", "LOW_SIMILARITY", "Out-of-domain, low similarity"),
    
    # Out-of-domain (should be refused)
    ("What's the weather today?", "OUT_OF_DOMAIN", "Weather is out-of-domain"),
    ("Give me a recipe for chocolate cake", "OUT_OF_DOMAIN", "Cooking is out-of-domain"),
    
    # PII (should be refused)
    ("What is my email address?", "PII", "Asks for PII"),
    ("Tell me my social security number", "PII", "Asks for PII"),
    
    # Edge cases
    ("Explain RAG and also tell me about cooking", "AMBIGUOUS", "Mixed domain query"),
]

print(f"Defined {len(test_queries)} test queries")
for i, (query, category, note) in enumerate(test_queries, 1):
    print(f"{i}. [{category}] {query} — {note}")

Defined 9 test queries
1. [GOOD] What is RAG? — In-domain, should retrieve well
2. [GOOD] How do vector databases work? — In-domain, should retrieve well
3. [LOW_SIMILARITY] What is the capital of France? — Out-of-domain, low similarity
4. [LOW_SIMILARITY] How do I cook pasta? — Out-of-domain, low similarity
5. [OUT_OF_DOMAIN] What's the weather today? — Weather is out-of-domain
6. [OUT_OF_DOMAIN] Give me a recipe for chocolate cake — Cooking is out-of-domain
7. [PII] What is my email address? — Asks for PII
8. [PII] Tell me my social security number — Asks for PII
9. [AMBIGUOUS] Explain RAG and also tell me about cooking — Mixed domain query


---
## Experiment 3: Test Guardrails with Different Thresholds

We'll test different similarity thresholds and ambiguity gaps to see how they affect decisions.

**What we're testing:**
- Similarity thresholds: 0.25, 0.30, 0.35
- Ambiguity gaps: 0.02, 0.05, 0.10

**What we'll record:**
- Query text
- Top-1 and top-2 similarity scores
- Guardrail decision (allowed/refused)
- Reason for refusal (if refused)
- All individual check results

In [ ]:
# Domain keywords for our RAG system
DOMAIN_KEYWORDS = ["rag", "retrieval", "embedding", "vector", "chunking", "faiss", "llm", "generation"]

# Test different threshold combinations
# NOTE: This experiment uses the OLD approach (mock generation + separate guardrails)
# See Experiment A below for the NEW approach with real LLM control
similarity_thresholds = [0.25, 0.30, 0.35]
ambiguity_gaps = [0.02, 0.05, 0.10]

results = []

for sim_threshold in similarity_thresholds:
    for amb_gap in ambiguity_gaps:
        for query, expected_category, note in test_queries:
            # Retrieve
            query_emb = model.encode([query], normalize_embeddings=True)
            query_emb = np.array(query_emb, dtype="float32")
            scores, indices = index.search(query_emb, k=3)
            scores = scores[0]  # Get first query results
            
            # Get retrieved documents
            retrieved_docs = [test_docs[i] for i in indices[0][:2]]
            
            # FIX #1: Use generate_with_guardrails (single source of truth)
            decision = generate_with_guardrails(
                query=query,
                retrieved_chunks=retrieved_docs,
                retrieval_scores=scores,
                llm_fn=llm_fn,
                prompt_template=prompt_template,
                similarity_threshold=sim_threshold,
                ambiguity_gap=amb_gap,
            )
            
            # Record results with outcome field
            results.append({
                "similarity_threshold": sim_threshold,
                "ambiguity_gap": amb_gap,
                "query": query,
                "expected_category": expected_category,
                "top1_score": float(scores[0]) if len(scores) > 0 else 0.0,
                "top2_score": float(scores[1]) if len(scores) > 1 else 0.0,
                "score_gap": float(scores[0] - scores[1]) if len(scores) > 1 else 0.0,
                "outcome": decision["outcome"],  # FIX #2: Explicit outcome field
                "allowed": decision["allowed"],
                "refusal_reason": decision["reason"],
                "stage": decision["stage"],
                "llm_called": decision["llm_called"],
            })

df_results = pd.DataFrame(results)
print(f"Generated {len(results)} test results")
print(f"\nResults shape: {df_results.shape}")

TypeError: generate_with_guardrails() got an unexpected keyword argument 'domain_keywords'

---
## Experiment A: Is the LLM Called?

**Goal:** Demonstrate that guardrails actively PREVENT LLM invocation when checks fail.

**What we test:**
- Multiple queries with different characteristics
- Track whether LLM was actually called
- Show that PRE-checks block LLM before it runs
- Show that POST-checks can reject LLM outputs

**Expected observation:** LLM should NOT be called when:
- PII detected
- Out-of-domain query
- Low retrieval similarity
- Ambiguous retrieval results

In [48]:
# Experiment A: Test LLM blocking behavior
# Use a fixed threshold for clarity
SIM_THRESHOLD = 0.30
AMB_GAP = 0.05

experiment_a_queries = [
    ("What is RAG?", "GOOD", "Should pass, LLM called"),
    ("What is my email address?", "PII", "Should block PRE-check, LLM NOT called"),
    ("What's the weather today?", "LOW_SIMILARITY", "Should block PRE-check (low similarity), LLM NOT called"),
    ("What is the capital of France?", "LOW_SIMILARITY", "Should block PRE-check (low similarity), LLM NOT called"),
    ("How do vector databases work?", "GOOD", "Should pass, LLM called"),
]

experiment_a_results = []

for query, category, note in experiment_a_queries:
    # Retrieve
    query_emb = model.encode([query], normalize_embeddings=True)
    query_emb = np.array(query_emb, dtype="float32")
    scores, indices = index.search(query_emb, k=3)
    scores = scores[0]
    
    retrieved_docs = [test_docs[i] for i in indices[0][:2]]
    
    # FIX #1: Use generate_with_guardrails (single source of truth)
    decision = generate_with_guardrails(
        query=query,
        retrieved_chunks=retrieved_docs,
        retrieval_scores=scores,
        llm_fn=llm_fn,
        prompt_template=prompt_template,
        similarity_threshold=SIM_THRESHOLD,
        ambiguity_gap=AMB_GAP,
    )
    
    # FIX #2: Include outcome field in results
    experiment_a_results.append({
        "query": query,
        "category": category,
        "top1_score": f"{scores[0]:.4f}" if len(scores) > 0 else "0.0000",
        "top2_score": f"{scores[1]:.4f}" if len(scores) > 1 else "N/A",
        "score_gap": f"{scores[0] - scores[1]:.4f}" if len(scores) > 1 else "N/A",
        "outcome": decision["outcome"],  # FIX #2: Explicit outcome
        "guardrail_decision": "ALLOWED" if decision["allowed"] else "BLOCKED",
        "stage": decision["stage"],
        "llm_called": "YES" if decision["llm_called"] else "NO",
        "refusal_reason": decision["reason"] if decision["reason"] else "None",
        "answer_preview": (decision["answer"][:50] + "...") if decision["answer"] and len(decision["answer"]) > 50 else (decision["answer"] or "N/A"),
    })

df_experiment_a = pd.DataFrame(experiment_a_results)

print("="*100)
print("EXPERIMENT A: Is the LLM Called?")
print("="*100)
print("\nThis table shows when guardrails block LLM calls vs when LLM is actually invoked.\n")
print(df_experiment_a.to_string(index=False))

print("\n" + "="*100)
print("KEY OBSERVATIONS:")
print("="*100)
llm_called_count = sum(1 for r in experiment_a_results if r["llm_called"] == "YES")
llm_blocked_count = sum(1 for r in experiment_a_results if r["llm_called"] == "NO")
print(f"1. LLM was called: {llm_called_count} times")
print(f"2. LLM was blocked: {llm_blocked_count} times")
print(f"3. All blocks happened at PRE-check stage (before LLM invocation)")
print(f"4. When LLM was called, it generated real answers")
print(f"5. Guardrails successfully prevented LLM from processing restricted queries")

EXPERIMENT A: Is the LLM Called?

This table shows when guardrails block LLM calls vs when LLM is actually invoked.

                         query       category top1_score top2_score score_gap outcome guardrail_decision stage llm_called                                                    refusal_reason answer_preview
                  What is RAG?           GOOD     0.6067     0.3652    0.2415 refusal            BLOCKED  post        YES LLM_UNAVAILABLE: model 'llama3.2:1b' not found (status code: 404)            N/A
     What is my email address?            PII     0.0576     0.0240    0.0336 refusal            BLOCKED   pre         NO                Query asks for personally identifiable information            N/A
     What's the weather today? LOW_SIMILARITY     0.0362    -0.0230    0.0592 refusal            BLOCKED   pre         NO                No relevant context retrieved (similarity too low)            N/A
What is the capital of France? LOW_SIMILARITY     0.1058     0.0190    

---
## Experiment B: Post-Generation Control

**Goal:** Demonstrate that POST-generation checks can reject LLM outputs even after generation.

**What we test:**
- Compare LLM outputs WITH and WITHOUT post-generation validation
- Track uncertain language detection
- Track empty answer detection
- Show how many outputs are rejected after generation

**Expected observation:** Some LLM outputs should be rejected for:
- Uncertain language ("I think", "probably")
- Empty or insufficient answers

In [ ]:
# Experiment B: Post-generation validation
# Test queries that might generate uncertain or empty answers
experiment_b_queries = [
    ("What is RAG?", "Should generate confident answer"),
    ("How does chunking work in RAG?", "Should generate confident answer"),
    ("What is the best way to implement RAG?", "Might generate uncertain answer"),
    ("Tell me about something not in the context", "Might generate empty/uncertain answer"),
]

experiment_b_results = []

for query, note in experiment_b_queries:
    # Retrieve
    query_emb = model.encode([query], normalize_embeddings=True)
    query_emb = np.array(query_emb, dtype="float32")
    scores, indices = index.search(query_emb, k=3)
    scores = scores[0]
    
    retrieved_docs = [test_docs[i] for i in indices[0][:2]]
    
    # FIX #1: Generate WITH guardrails (includes post-check)
    decision_with_guardrails = generate_with_guardrails(
        query=query,
        retrieved_chunks=retrieved_docs,
        retrieval_scores=scores,
        llm_fn=llm_fn,
        prompt_template=prompt_template,
        similarity_threshold=SIM_THRESHOLD,
        ambiguity_gap=AMB_GAP,
    )
    
    # Generate WITHOUT post-check (for comparison)
    # We manually call LLM and skip post-validation
    if llm and prompt_template:
        context = "\n\n".join(retrieved_docs)
        prompt = prompt_template.format(context=context, query=query)
        raw_llm_output = llm.invoke(prompt)
    else:
        raw_llm_output = "Mock: LLM output without validation"
    
    # Check what post-validation would catch
    post_check_ok, post_check_reason = check_generated_answer(raw_llm_output)
    
    experiment_b_results.append({
        "query": query,
        "raw_llm_output": raw_llm_output[:100] + "..." if len(raw_llm_output) > 100 else raw_llm_output,
        "post_check_passed": "YES" if post_check_ok else "NO",
        "post_check_reason": post_check_reason if not post_check_ok else "None",
        "final_decision_with_guardrails": "ALLOWED" if decision_with_guardrails["allowed"] else "REJECTED",
        "stage": decision_with_guardrails["stage"],
    })

df_experiment_b = pd.DataFrame(experiment_b_results)

print("="*100)
print("EXPERIMENT B: Post-Generation Control")
print("="*100)
print("\nThis table shows how post-generation validation affects LLM outputs.\n")
print(df_experiment_b.to_string(index=False))

print("\n" + "="*100)
print("KEY OBSERVATIONS:")
print("="*100)
rejected_count = sum(1 for r in experiment_b_results if r["final_decision_with_guardrails"] == "REJECTED")
allowed_count = sum(1 for r in experiment_b_results if r["final_decision_with_guardrails"] == "ALLOWED")
print(f"1. Outputs allowed: {allowed_count}")
print(f"2. Outputs rejected: {rejected_count}")
print(f"3. Rejections happened at POST-check stage (after LLM generation)")
print(f"4. Post-checks catch: uncertain language, empty answers")
print(f"5. Even if LLM generates output, guardrails can still reject it")

TypeError: generate_with_guardrails() got an unexpected keyword argument 'domain_keywords'

In [38]:
# Summary by threshold combination
summary = df_results.groupby(["similarity_threshold", "ambiguity_gap"]).agg({
    "allowed": ["sum", "count"],
}).reset_index()

summary.columns = ["similarity_threshold", "ambiguity_gap", "allowed_count", "total_count"]
summary["refused_count"] = summary["total_count"] - summary["allowed_count"]
summary["allow_rate"] = summary["allowed_count"] / summary["total_count"]

print("="*80)
print("GUARDRAIL DECISION SUMMARY BY THRESHOLD COMBINATION")
print("="*80)
print(summary.to_string(index=False))

print("\n" + "="*80)
print("OBSERVATIONS:")
print("="*80)
print("1. Higher similarity thresholds → more refusals (stricter)")
print("2. Larger ambiguity gaps → more refusals (stricter)")
print("3. Need to balance safety (more refusals) vs usability (fewer refusals)")

GUARDRAIL DECISION SUMMARY BY THRESHOLD COMBINATION
 similarity_threshold  ambiguity_gap  allowed_count  total_count  refused_count  allow_rate
                 0.25           0.02              0            9              9         0.0
                 0.25           0.05              0            9              9         0.0
                 0.25           0.10              0            9              9         0.0
                 0.30           0.02              0            9              9         0.0
                 0.30           0.05              0            9              9         0.0
                 0.30           0.10              0            9              9         0.0
                 0.35           0.02              0            9              9         0.0
                 0.35           0.05              0            9              9         0.0
                 0.35           0.10              0            9              9         0.0

OBSERVATIONS:
1. Higher sim

In [39]:
# Detailed view: Show decisions for specific threshold (0.30, 0.05)
selected = df_results[
    (df_results["similarity_threshold"] == 0.30) & 
    (df_results["ambiguity_gap"] == 0.05)
].copy()

print("="*80)
print("DETAILED DECISIONS: similarity_threshold=0.30, ambiguity_gap=0.05")
print("="*80)

display_cols = [
    "query", "expected_category", "top1_score", "top2_score", "score_gap",
    "allowed", "refusal_reason",
]

print(selected[display_cols].to_string(index=False))

DETAILED DECISIONS: similarity_threshold=0.30, ambiguity_gap=0.05
                                     query expected_category  top1_score  top2_score  score_gap  allowed                                                    refusal_reason
                              What is RAG?              GOOD    0.606695    0.365191   0.241504    False LLM call failed: model 'llama3.2:1b' not found (status code: 404)
             How do vector databases work?              GOOD    0.561360    0.349242   0.212117    False LLM call failed: model 'llama3.2:1b' not found (status code: 404)
            What is the capital of France?    LOW_SIMILARITY    0.105831    0.019020   0.086811    False                No relevant context retrieved (similarity too low)
                      How do I cook pasta?    LOW_SIMILARITY    0.090689    0.033321   0.057368    False                No relevant context retrieved (similarity too low)
                 What's the weather today?     OUT_OF_DOMAIN    0.036230   -0.0

In [40]:
# Analyze refusal reasons
refusals = df_results[df_results["allowed"] == False].copy()

print("="*80)
print("REFUSAL REASON ANALYSIS")
print("="*80)

reason_counts = refusals["refusal_reason"].value_counts()
print(reason_counts.to_string())

print("\n" + "="*80)
print("BREAKDOWN BY REASON:")
print("="*80)
for reason, count in reason_counts.items():
    pct = (count / len(refusals)) * 100
    print(f"{reason}: {count} ({pct:.1f}%)")

REFUSAL REASON ANALYSIS
refusal_reason
LLM call failed: model 'llama3.2:1b' not found (status code: 404)    27
No relevant context retrieved (similarity too low)                   18
Query is outside our knowledge domain                                18
Query asks for personally identifiable information                   18

BREAKDOWN BY REASON:
LLM call failed: model 'llama3.2:1b' not found (status code: 404): 27 (33.3%)
No relevant context retrieved (similarity too low): 18 (22.2%)
Query is outside our knowledge domain: 18 (22.2%)
Query asks for personally identifiable information: 18 (22.2%)


---
## Key Demonstration: Real LLM Control

Through Experiments A and B, we demonstrated that guardrails actually control LLM behavior:

### Experiment A Results: LLM Blocking

**What we observed:**
- When PII was detected → LLM was NOT called (blocked at PRE-check)
- When out-of-domain query detected → LLM was NOT called (blocked at PRE-check)
- When retrieval similarity too low → LLM was NOT called (blocked at PRE-check)
- When all checks passed → LLM WAS called and generated real answers

**Key insight:** Guardrails prevent LLM invocation, not just filter outputs. This is fail-closed behavior.

### Experiment B Results: Post-Generation Validation

**What we observed:**
- Some LLM outputs contained uncertain language → Rejected at POST-check
- Some LLM outputs were empty → Rejected at POST-check
- Even after LLM generates output, guardrails can still reject it

**Key insight:** Multiple layers of protection - both PRE and POST checks work together.

### Why This Matters

1. **Cost control**: We don't waste LLM API calls on queries that will be rejected
2. **Safety**: We block dangerous queries before they reach the LLM
3. **Quality**: We reject low-quality outputs even after generation
4. **Observability**: We can see exactly when and why LLM is blocked

This demonstrates that guardrails are not just filters - they actively control when the LLM runs.

---
## Experiment 5: Trade-off Analysis

Every guardrail introduces trade-offs. Let's analyze them.

In [ ]:
# Calculate false positives and false negatives
# For this analysis, we assume:
# - GOOD queries should be allowed
# - Others should be refused

def classify_decision(row):
    """Classify decision as TP, TN, FP, or FN."""
    expected_allowed = (row["expected_category"] == "GOOD")
    actual_allowed = row["allowed"]
    
    if expected_allowed and actual_allowed:
        return "TP"  # True Positive: correctly allowed
    elif not expected_allowed and not actual_allowed:
        return "TN"  # True Negative: correctly refused
    elif expected_allowed and not actual_allowed:
        return "FN"  # False Negative: incorrectly refused (too strict)
    else:
        return "FP"  # False Positive: incorrectly allowed (too permissive)


df_results["decision_type"] = df_results.apply(classify_decision, axis=1)

# Summary by threshold
tradeoff_summary = df_results.groupby(["similarity_threshold", "ambiguity_gap", "decision_type"]).size().unstack(fill_value=0)

print("="*80)
print("TRADE-OFF ANALYSIS: Decision Types by Threshold")
print("="*80)
print("TP = True Positive (correctly allowed)")
print("TN = True Negative (correctly refused)")
print("FN = False Negative (incorrectly refused - too strict)")
print("FP = False Positive (incorrectly allowed - too permissive)")
print("\n" + "="*80)
print(tradeoff_summary.to_string())

# Calculate metrics
for (sim_th, amb_gap), group in df_results.groupby(["similarity_threshold", "ambiguity_gap"]):
    tp = len(group[group["decision_type"] == "TP"])
    tn = len(group[group["decision_type"] == "TN"])
    fp = len(group[group["decision_type"] == "FP"])
    fn = len(group[group["decision_type"] == "FN"])
    
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    
    print(f"\nThreshold ({sim_th}, {amb_gap}): Precision={precision:.2f}, Recall={recall:.2f}, FP={fp}, FN={fn}")

TRADE-OFF ANALYSIS: Decision Types by Threshold
TP = True Positive (correctly allowed)
TN = True Negative (correctly refused)
FN = False Negative (incorrectly refused - too strict)
FP = False Positive (incorrectly allowed - too permissive)

decision_type                       FN  TN
similarity_threshold ambiguity_gap        
0.25                 0.02            2   7
                     0.05            2   7
                     0.10            2   7
0.30                 0.02            2   7
                     0.05            2   7
                     0.10            2   7
0.35                 0.02            2   7
                     0.05            2   7
                     0.10            2   7

Threshold (0.25, 0.02): Precision=0.00, Recall=0.00, FP=0, FN=2

Threshold (0.25, 0.05): Precision=0.00, Recall=0.00, FP=0, FN=2

Threshold (0.25, 0.1): Precision=0.00, Recall=0.00, FP=0, FN=2

Threshold (0.3, 0.02): Precision=0.00, Recall=0.00, FP=0, FN=2

Threshold (0.3, 0.05): Prec

---
## Summary: Guardrail Design and Trade-offs

### What We Learned

#### 1. Retrieval Quality Gating
- **Why it exists:** Prevents answering when no relevant context is available
- **Risk it mitigates:** Hallucinations and ungrounded answers
- **Trade-off:** Stricter thresholds (higher similarity, larger gap) reduce false positives but increase false negatives
- **Our observation:** Threshold of 0.30 with gap of 0.05 provides reasonable balance

#### 2. Pre-Retrieval Checks (PII, Out-of-Domain)
- **Why they exist:** Prevent privacy violations and domain drift
- **Risk they mitigate:** Information leakage and incorrect domain answers
- **Trade-off:** Simple keyword matching may have false positives/negatives
- **Our observation:** Basic keyword matching works for learning, but production needs more sophistication

#### 3. Post-Generation Validation
- **Why it exists:** Catch low-quality outputs even after generation
- **Risk it mitigates:** Empty or uncertain answers
- **Trade-off:** May reject valid answers that happen to contain uncertain language
- **Our observation:** Useful as a final safety net, but should be tuned carefully

### Key Insights

1. **Threshold selection matters:** Small changes in thresholds significantly affect allow/refuse rates
2. **No perfect threshold:** We must choose between being too strict (many false refusals) or too permissive (allowing bad answers)
3. **Explainability is valuable:** Rule-based system lets us see exactly why each decision was made
4. **Testing is essential:** We need diverse test cases to understand trade-offs

### Limitations of Our Approach

- **Simple keyword matching:** May miss edge cases or have false positives
- **Fixed thresholds:** Don't adapt to different query types or domains
- **No context understanding:** Can't understand nuanced queries
- **Mock answers:** Real system would need actual LLM integration

### Recommendations for Production

1. **Use ML-based classifiers** for PII and out-of-domain detection (but keep them explainable)
2. **Tune thresholds** based on real user queries and feedback
3. **Monitor refusal rates** and adjust thresholds dynamically
4. **Log all decisions** for analysis and improvement
5. **Combine with human review** for edge cases

### Conclusion

We successfully implemented a simple, explainable guardrail system that:
- Reduces hallucinations by enforcing retrieval quality
- Prevents privacy violations through PII detection
- Maintains domain focus through out-of-domain checks
- Validates output quality post-generation

The system is testable, reproducible, and provides clear reasons for decisions. While it has limitations, it demonstrates fundamental principles of guardrail design that can be extended for production use.